# First-Year Writing Seminars

**Objectives:**
* Assign students to FWS sections so that they get one of their top 5 choices.
* Improve assignments by making changes to our transportation formulation.

**Key Ideas:**
* integrality property
* the transportation problem
* the assignment problem

**Brief description:** If you recall pre-enroll, there was a separate ballot you completed by listing your top 5 picks for FWS that semester. You were later notified which class you got placed into, probably hoping it was your first choice. By now, this should not seem like magic; problems like these often enlist help from Operations Research especially as the scale increases. Disclaimer: the following model is not actually used by Cornell.

In [ ]:
# imports -- make sure to run this cell
import pandas as pd
import math, itertools
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite
from ortools.linear_solver import pywraplp as OR

## Part 1: Brainstorming

We want to *assign* a class to each student. This sounds like an assignment problem, which is a special case of the *transportation problem*. 

**Example 1**  
The following table gives an instance with 7 students (1-7) and 4 classes (A-D). Each class has a capacity of 2 students. Column `1` gives the student's first preference and `2` gives their second preference.

In [ ]:
fws_7_students = pd.read_csv('data/fws_7_students.csv', index_col=0)
display(fws_7_students)

**Q1:** What are the supply nodes?  

**A:**   

**Q2:** What are the supply values of the supply nodes? (Hint: How many units can be transferred?)

**A:**   

**Q3:** What are the demand nodes?  

**A:**   

**Q4:** What are the demand values of the demand nodes? (Hint: How many units can be received?)

**A:**   

**Q5:** What does a directed edge from a supply node to a demand node indicate?  

**A:** 

**Q6:** What are the edge costs?  

**A:** 

**Q7:** Visualize your graph by completing the dictionary of edge costs. This will be used later as the input to the LP model.

In [ ]:
from fws_lab import ex1

# TODO: Uncomment and complete the dictionary of edge costs
# S = [1,2,3,4,5,6,7]
# D = ['A','B','C','D']
# E = {(1,'A'):1, 
#      (1,'B'):2, 
#      (2,'D'): , 
#      (2,'C'): , 
#      (3,'A'): , 
#      (3,'C'): , 
#      (4,'B'): , 
#      (4,'D'): , 
#      (5,'C'): , 
#      (5,'B'): , 
#      (6,'A'): , 
#      (6,'B'): , 
#      (7,'B'): , 
#      (7,'A'):  }



ex1(S, D, E)

To complete the simple model, we need to answer one more question.  

**Q8:** The total demand is 8, but the current total supply is only 7. How can we make sure that demand is satisfied for there to be a feasible solution?  

**A:**   

**Q9:** Create a similar list of students, classes, and edge weights like the lists above but account for the dummy supply node (which is indexed as zero).

In [ ]:
# TODO: Uncomment and complete the dictionary of edge costs
# S_dummy = [1,2,3,4,5,6,7,0]
# D_dummy = ['A','B','C','D']
# E_dummy = {(1,'A'):1, (1,'B'):2, (2,'D'):1, (2,'C'):2, (3,'A'):1, (3,'C'):2, 
#            (4,'B'):1, (4,'D'):2, (5,'C'):1, (5,'B'):2, (6,'A'):1, (6,'B'):2, 
#            (7,'B'):1, (7,'A'):2, (0,'A'): XXX, (0,'B'): XXX, (0,'C'): XXX, (0,'D'): XXX}



ex1(S_dummy, D_dummy, E_dummy)

## Part 2: Solving

Let's use OR-Tools to define our mathematical model. 

**Q10:** Complete the constraints in the cell below.

In [ ]:
def fws(preferences, capcity, penalty, integer = False):
    """A model for solving a first-year writing seminar assignment problem.
    
    Args:
        preferences (pd.DataFrame): Preferred classes for each student.
        capcity (int): Capacity of the classroom.
        penalty (float): Penalty for not assigning a student to one of their top 5 picks.
    """
    STUDENTS = list(preferences.index) + ['dummy']             # students
    
    CLASSES = []
    for c in preferences.columns:
        CLASSES = CLASSES + list(preferences[c].unique())
    CLASSES = list(set(CLASSES))                               # classes
    
    edge_costs = {}                                 
    for s in preferences.index:
        for c in preferences:
            edge_costs[(s, preferences.at[int(s),c])] = int(c)
    EDGES = list(edge_costs)                                   # edges
    
    # add dummy edges
    dummy_edges = list(itertools.product(['dummy'], CLASSES))
    EDGES.extend(dummy_edges)          
    
    c = edge_costs                                             # costs
    # add dummy edge costs
    for edge in dummy_edges:
        c.update({edge : penalty})    
    
    # define model
    m = OR.Solver('fws', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}    
    for i,j in EDGES:
        if integer:
            x[i,j] = m.IntVar(0, m.infinity(), ('(%s, %s)' % (i,j)))
        else:
            x[i,j] = m.NumVar(0, m.infinity(), ('(%s, %s)' % (i,j)))
        
    # objective function
    m.Minimize(sum(c[i,j]*x[i,j] for i,j in EDGES))
       
    # subject to: each student (not including the dummy) is assigned at most one class
    for k in STUDENTS:
        if k != 'dummy':
            m.Add(sum(x[i,j] for i,j in EDGES if i==k) <= 1)
        
    # subject to: each class is full
    # TODO: Write the constraint that each class is full
    
    
    
    return m,x

In [ ]:
def solution_summary(preferences, penalty, m):
    """Print a summary of the solution."""
    students = list(preferences.index) + ['dummy']

    classes = []
    for c in preferences.columns:
        classes = classes + list(preferences[c].unique())
    classes = list(set(classes))   

    edge_costs = {}                                 
    for s in preferences.index:
        for c in preferences:
            edge_costs[(s, preferences.at[int(s),c])] = int(c)
    edges = list(edge_costs)

    dummy_edges = list(itertools.product(['dummy'], classes))
    edges.extend(dummy_edges)          

    c = edge_costs
    for edge in dummy_edges:
        c.update({edge : penalty}) 
        
    unmatched = []
    for k in students:
        if (sum(x[i,j].solution_value() for i,j in edges if i==k) == 0) and (k!='dummy'):
            unmatched.append(k)
    print("Unmatched students:", len(unmatched))
    
    matched = {}
    for i,j in edges:
        if x[i,j].solution_value() == 1:
            if c[i,j] in matched:
                matched[c[i,j]] += 1
            else:
                matched.update({c[i,j] : 1})
    
    if penalty in matched.keys():
        del matched[penalty]
        
    return matched

**Q11:** Replace `XXX` with the cost of the dummy edges and then run the cell.

In [ ]:
# TODO: Uncomment and replace XXX with the cost of the dummy edges
# m,x = fws(fws_7_students, 2, XXX)
# m.Solve()
# solution_summary(fws_7_students, XXX,m)



6 students got their first choice, 1 student got their second choice, and no students were left unmatched.   

**Q12:** Why can't all the students be assigned their first choice? Or do you think the answer you got could be better?  

**A:**  

**Example 2**  
In this new instance with 8 students (1-8) and 4 classes (A-D), more students prefer A and B than C and D.

In [ ]:
fws_8_students = pd.read_csv('data/fws_8_students.csv', index_col=0)
display(fws_8_students)

In [ ]:
from fws_lab import ex2

S2 = [1,2,3,4,5,6,7,8]
D2 = ['A','B','C','D']
E2 = {(1,'A'):1, (1,'B'):2, (2,'B'):1, (2,'A'):2, (3,'C'):1, (3,'D'):2, (4,'A'):1, (4,'B'):2, 
      (5,'B'):1, (5,'A'):2, (6,'A'):1, (6,'C'):2, (7,'C'):1, (7,'D'):2, (8,'A'):1, (8,'D'):2}

ex2(S2, E2)

In [ ]:
# cost of edges from dummy node is 2.5
m,x = fws(fws_8_students, 2, 2.5)
m.Solve()
solution_summary(fws_8_students, 2.5, m)

There is 1 unmatched student when using 2.5 as the cost of dummy edges.  

**Q13:** Why can't all the students be assigned one of their top 2 choices? Or do you think the answer you got could be better?  

**A:**   

Re-solve using 3.5 as the cost.

In [ ]:
# cost of edges from dummy node is 3.5
m,x = fws(fws_8_students, 2, 3.5)
m.Solve()
solution_summary(fws_8_students, 3.5, m)

**Q14:** Both the edge weights 2.5 and 3.5 are greater than 2. In some sense, they are prioritized last. We rather send flow across 1 and 2 weight edges. However, we get different solutions. Why?

**A:**   

When solving the actual data, you will see that other subtle reasons might lead to unmatched students.

## Part 3: Solving the Actual Data

There are 2886 students and 183 class sections. Assume each class can have at most 16 students. As we already know, each student picks their top 5 classes.

In [ ]:
fws_f09_ballots = pd.read_csv('data/fws_f09_ballots.csv', index_col=0)
fws_f09_ballots.head()

There are 6 columns with the first being student # and the other 5 being first, second, third, fourth, and fifth choice. Each row is a student, and class # indicates the class picked as the choice belonging to the column.

In [ ]:
m,x = fws(fws_f09_ballots, 16, 6)
m.Solve()
solution_summary(fws_f09_ballots, 6, m)

We got an answer! Unfortunately, there are 107 students who did not get any of their top 5 picks. Let's try to make our model better, so that no students are unmatched.  

The objective function is actually a weighted function. The coefficients, which in this case are the costs, dictate how much you want the solver to select the corresponding edges. An edge with a small cost has a higher likelihood (weight) of being in the solution while an edge with a large cost will potentially be avoided. This being said, we can try setting the cost of edges from the dummy node to an arbitrarily large number like 100,000. This should create greater incentive to fill classes with actual students than our fake filler 'students' as we witnessed in the second small example.

In [ ]:
m,x = fws(fws_f09_ballots, 16, 100000)
m.Solve()
sol = solution_summary(fws_f09_ballots, 100000, m)
sol

Yay! We just improved our model. As a bonus, we can look at the distribution of preferences received by students in the form of a histogram and see how our solution did in percentages.

In [ ]:
from fws_lab import Histo

Histo(sol, 15)
for pref in range(1,6):
    print(pref, ":", round(100*sol[pref]/2886, 2), "%")

## Bonus 1

We do not want to waste time trying different cost combinations if there is no solution where every student gets one of their top 5 picks. How can we check whether there exists a feasible solution with 0 unmatched students?  

**B1:** What is the cost of edges representing students' preferences?  

**A:**   

**B2:** What is the cost of dummy edges?  
    
**A:**   

**B3:** What is the desired solution?  

**A:**   

**B4:** Describe another way that might use a different model.  

**A:**   

## Bonus 2

There are many ways to improve a model even if the type of problem does not change. Previously, we decided it was important for an assignment to guarantee that everyone got at least one of their top 5 picks. How can you make a better yet model? 

Your answer will depend on what you think makes the best assignment. Some things to consider: reducing the number of students getting their lesser-preferred choices, adding a lower bound on class size, choosing for two students to both have their 3rd choice instead of one 1st and the other 5th, using recursion...